In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import torch
print("pytorch ver . ",torch.__version__)
print("Is CUDA available?",torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.utils.data as Data
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import xarray as xr
import os
os.environ['R_HOME'] = '/home/luhung3080/miniconda3/envs/chou/lib/R'
from rpy2.robjects import r, numpy2ri
numpy2ri.activate()
from rpy2.robjects.packages import importr
sinkr = importr('sinkr')
import Nio
print ("pynio ver . ",Nio.__version__)

/home/luhung3080/miniconda3/envs/chou/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pytorch ver .  1.11.0+cu113
Is CUDA available? True
pynio ver .  1.5.5


# Y = b0 + b1X12

## Data

In [2]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/data_final.csv')
data

,SITE_ID,date,FCST_TIME,TAU,pm25_cal,pm25_obs
0,EPA001,2020-02-24 08:00:00,2020-02-24 09:00:00,1,4.9510,10.0
1,EPA001,2020-02-24 08:00:00,2020-02-24 10:00:00,2,4.4674,13.0
2,EPA001,2020-02-24 08:00:00,2020-02-24 11:00:00,3,4.6159,11.0
3,EPA001,2020-02-24 08:00:00,2020-02-24 12:00:00,4,3.9937,11.0
4,EPA001,2020-02-24 08:00:00,2020-02-24 13:00:00,5,3.9602,9.0
...,...,...,...,...,...,...
3092755,EPA080,2021-10-30 08:00:00,2021-11-02 04:00:00,68,3.6190,4.0
3092756,EPA080,2021-10-30 08:00:00,2021-11-02 05:00:00,69,3.7908,7.0
3092757,EPA080,2021-10-30 08:00:00,2021-11-02 06:00:00,70,4.0454,7.0
3092758,EPA080,2021-10-30 08:00:00,2021-11-02 07:00:00,71,3.9015,4.0


## 一開始就把x1 x2分開讀取，補完再用x1推y

In [3]:
x1=np.zeros([604,1704])
x2=np.zeros([604,5112])

#x1 (0th~1703th column as x)
for i in range (0,604):
    for j in range (0,71):
        a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
        for k in range (0,24):
            x1[i][j*24+k]=a[k]

#x2 (1704th~8519th column as x)
for i in range (1,604):
    b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    for j in range(0,5112):
        x2[i-1][j]=b[j]
        
print(np.shape(x1))
print(np.shape(x2))

(604, 1704)
(604, 5112)


In [4]:
x1Restruct_Fun=x1
x2Restruct_Fun=x2
print(np.shape(x1Restruct_Fun))
print(np.shape(x2Restruct_Fun))

(604, 1704)
(604, 5112)


In [5]:
YRestruct_Fun=np.zeros([601,5112])
for j in range (0,71):
    for i in range(0,601):
        YRestruct_Fun[i][72*j:72*j+24]=x1Restruct_Fun[1+i][24*j:24*j+24]
        YRestruct_Fun[i][72*j+24:72*j+48]=x1Restruct_Fun[1+i+1][24*j:24*j+24]
        YRestruct_Fun[i][72*j+48:72*j+72]=x1Restruct_Fun[1+i+2][24*j:24*j+24]
print(np.shape(YRestruct_Fun))

(601, 5112)


In [6]:
XRestruct_Fun=np.zeros([601,6816])
for i in range (0,601):
    for j in range (0,1704):
        XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
    for j in range (1704,6816):
        XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]
print(np.shape(XRestruct_Fun))

(601, 6816)


In [7]:
Xhat=np.zeros([540,6816])
Yhat=np.zeros([540,5112])
for i in range (0,540):
    for j in range (0,5112):
        Xhat[i][j]=XRestruct_Fun[i][j]
        Yhat[i][j]=YRestruct_Fun[i][j]
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))

Xhat
(540, 6816)
Yhat
(540, 5112)


In [8]:
print(np.linalg.matrix_rank(Xhat))
print(np.linalg.matrix_rank(Yhat))

540
540


## PCA95%

In [25]:
pca95 = PCA(n_components=0.95)
pca95.fit(Xhat)

PCA(n_components=0.95)

In [26]:
x_pca95=pca95.transform(Xhat)
print(x_pca95.shape)
print(np.linalg.matrix_rank(x_pca95))

(540, 180)
180


### LinearRegression

In [27]:
model = LinearRegression()
model.fit(x_pca95, Yhat)
print(np.shape(model.coef_))

(5112, 180)


In [28]:
fit95=model.predict(x_pca95)
print(fit95.shape)

(540, 5112)


### compare (training data)

In [29]:
data0=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')
dd = data0.drop(data0[data0['TAU'] == 0].index)

In [30]:
fit95_test=fit95
"""
old data
"""
old_x=np.zeros([601,5112])
old_y=np.zeros([601,5112])
#data_cal
for i in range (1,602):
    a=np.array(dd['VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
    old_x[i-1]=a
#data_obs
for i in range (0,601):
    a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    old_y[i-1]=a
print(np.shape(old_x))
print(np.shape(old_y))
print(np.shape(fit95_test))

(601, 5112)
(601, 5112)
(540, 5112)


In [31]:
old_x1=np.zeros([540,5112])
old_y1=np.zeros([540,5112])
fit_fx1=np.zeros([540,5112])
for i in range (0,540):
    for j in range (0,5112):
        old_x1[i][j]=old_x[i][j]
        old_y1[i][j]=old_y[i][j]
        fit_fx1[i][j]=fit95_test[i][j]
before1=abs(old_x1-old_y1)
after1=abs(fit_fx1-old_y1)
before_error1=np.nanmean(before1)
after_error1=np.nanmean(after1)
print("January to August (training data set) before_error (MADE) of 95 : %f" %before_error1)
print("January to August (training data set) after_error (MADE) of 95 : %f" %after_error1)

January to August (training data set) before_error (MADE) of 95 : 7.174635
January to August (training data set) after_error (MADE) of 95 : 3.700996


### compare (testing data) 

In [33]:
"""
generate new cal
"""
x1=np.zeros([604,1704])
x2=np.zeros([604,5112])

#x1 (0th~1703th column as x)
for i in range (0,604):
    for j in range (0,71):
        a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
        for k in range (0,24):
            x1[i][j*24+k]=a[k]

#x2 (1704th~6816th column as x)
for i in range (1,604):
    b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    for j in range(0,5112):
        x2[i-1][j]=b[j]

x1Restruct_Fun=x1
x2Restruct_Fun=x2
XRestruct_Fun=np.zeros([601,6816])
for i in range (0,601):
    for j in range (0,1704):
        XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
    for j in range (1704,6816):
        XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]
print(np.shape(XRestruct_Fun))

xf = np.zeros([61,6816])
for i in range (0,61):
    for j in range(0,6816):
        xf[i][j] = XRestruct_Fun[i+540][j]

new_xcal=pca95.transform(xf)
new_xfit=model.predict(new_xcal)
print(np.shape(new_xfit))

(601, 6816)
(61, 5112)


In [34]:
"""
old data
"""
old_x=np.zeros([601,5112])
old_y=np.zeros([601,5112])
#data_cal
for i in range (1,602):
    a=np.array(dd['VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
    old_x[i-1]=a
#data_obs
for i in range (1,602):
    a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    old_y[i-1]=a
print(np.shape(old_x))
print(np.shape(old_y))

(601, 5112)
(601, 5112)


In [35]:
old_x2=np.zeros([61,5112])
old_y2=np.zeros([61,5112])
fit_fx2=np.zeros([61,5112])
for i in range (0,61):
    for j in range (0,5112):
        old_x2[i][j]=old_x[i+540][j]
        old_y2[i][j]=old_y[i+540][j]
        fit_fx2[i][j]=new_xfit[i][j]
before2=abs(old_x2-old_y2)
after2=abs(fit_fx2-old_y2)
before_error2=np.nanmean(before2)
after_error2=np.nanmean(after2)
print("September (testing data set) before_error (MADE) of 95 : %f" %before_error2)
print("September (testing data set) after_error (MADE) of 95 : %f" %after_error2)

September (testing data set) before_error (MADE) of 95 : 6.989377
September (testing data set) after_error (MADE) of 95 : 5.718768


In [36]:
per = np.zeros(99)
for i in range (0,99):
    per[i]=0.01*(i+1)
print(per)
per_error = np.zeros(99)

[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13 0.14
 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27 0.28
 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42
 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51 0.52 0.53 0.54 0.55 0.56
 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7
 0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79 0.8  0.81 0.82 0.83 0.84
 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98
 0.99]


In [9]:
for z in range (0,99):
    print(z+1)
    ## PCA90%
    pca90 = PCA(n_components=per[z])
    pca90.fit(Xhat)
    #
    x_pca90=pca90.transform(Xhat)
    print(np.linalg.matrix_rank(x_pca90))
    ### LinearRegression
    model = LinearRegression()
    model.fit(x_pca90, Yhat)
    print(np.shape(model.coef_))
    #
    fit90=model.predict(x_pca90)
    ### compare (training data)
    fit90_test=fit90
    """
    old data
    """
    old_x=np.zeros([601,5112])
    old_y=np.zeros([601,5112])
    #data_cal
    for i in range (1,602):
        a=np.array(dd['VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
        old_x[i-1]=a
    #data_obs
    for i in range (0,601):
        a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
            if a[j]=='A' :
                a[j]=np.nan
            if a[j]=='*' :
                a[j]=np.nan
            if a[j]=='x' :
                a[j]=np.nan  
        old_y[i-1]=a
    #
    old_x1=np.zeros([540,5112])
    old_y1=np.zeros([540,5112])
    fit_fx1=np.zeros([540,5112])
    for i in range (0,540):
        for j in range (0,5112):
            old_x1[i][j]=old_x[i][j]
            old_y1[i][j]=old_y[i][j]
            fit_fx1[i][j]=fit90_test[i][j]
    before1=abs(old_x1-old_y1)
    after1=abs(fit_fx1-old_y1)
    before_error1=np.nanmean(before1)
    after_error1=np.nanmean(after1)
    ### compare (testing data)
    """
    generate new cal
    """
    x1=np.zeros([604,1704])
    x2=np.zeros([604,5112])

    #x1 (0th~1703th column as x)
    for i in range (0,604):
        for j in range (0,71):
            a=np.array(data['pm25_obs'][5112*i+72*j:5112*i+72*j+24])
            for k in range (0,24):
                x1[i][j*24+k]=a[k]

    #x2 (1704th~6816th column as x)
    for i in range (1,604):
        b=np.array(data['pm25_cal'][5112*i:5112*i+5112])
        for j in range(0,5112):
            x2[i-1][j]=b[j]

    x1Restruct_Fun=x1
    x2Restruct_Fun=x2
    XRestruct_Fun=np.zeros([601,6816])
    for i in range (0,601):
        for j in range (0,1704):
            XRestruct_Fun[i][j]=x1Restruct_Fun[i][j]
        for j in range (1704,6816):
            XRestruct_Fun[i][j]=x2Restruct_Fun[i][j-1704]

    xf = np.zeros([61,6816])
    for i in range (0,61):
        for j in range(0,6816):
            xf[i][j] = XRestruct_Fun[i+540][j]

    new_xcal=pca90.transform(xf)
    new_xfit=model.predict(new_xcal)

    #
    """
    old data
    """
    old_x=np.zeros([601,5112])
    old_y=np.zeros([601,5112])
    #data_cal
    for i in range (1,602):
        a=np.array(dd['VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
        old_x[i-1]=a
    #data_obs
    for i in range (1,602):
        a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
        for j in range(0,5112):
            if a[j]=='NaN' :
                a[j]=np.nan
            if a[j]=='A' :
                a[j]=np.nan
            if a[j]=='*' :
                a[j]=np.nan
            if a[j]=='x' :
                a[j]=np.nan  
        old_y[i-1]=a
    #
    old_x2=np.zeros([61,5112])
    old_y2=np.zeros([61,5112])
    fit_fx2=np.zeros([61,5112])
    for i in range (0,61):
        for j in range (0,5112):
            old_x2[i][j]=old_x[i+540][j]
            old_y2[i][j]=old_y[i+540][j]
            fit_fx2[i][j]=new_xfit[i][j]
    before2=abs(old_x2-old_y2)
    after2=abs(fit_fx2-old_y2)
    before_error2=np.nanmean(before2)
    after_error2=np.nanmean(after2)    
    print("January to August (training data set) before_error (MADE) of 90 : %f" %before_error1)
    print("January to August (training data set) after_error (MADE) of 90 : %f" %after_error1)
    print("September (testing data set) before_error (MADE) of 90 : %f" %before_error2)
    print("September (testing data set) after_error (MADE) of 90 : %f" %after_error2)
    per_error[z] = after_error2

1


NameError: name 'per' is not defined

In [38]:
print(per_error)

[6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.3466428  6.3466428  6.3466428  6.3466428  6.3466428  6.3466428
 6.17989408 6.17989408 6.17989408 6.17989408 6.17989408 6.17989408
 6.17989408 6.17989408 6.17989408 6.17989408 6.17989408 6.17989408
 6.38954351 6.38954351 6.38954351 6.38954351 6.38954351 6.38954351
 6.38954351 6.22348962 6.22348962 6.22348962 6.22348962 6.08456267
 6.08456267 6.07643685 6.07643685 6.07643685 5.89402931 5.68002756
 5.68002756 5.62004936 5.63702169 5.62717568 5.62302511 5.62302511
 5.57182115 5.52282439 5.45632312 5.43828866 5.43359993 5.44846571
 5.43481479 5.31026027 5.25796781 5.22436342 5.22722523 5.3165738
 5.36139257 5.3549918  5.33443709 5.37461059 5.33616535 5.34546282
 5

In [39]:
print(0.01*(np.argmin(per_error)+1))
print(min(per_error))

0.8200000000000001
5.224363416035845
